# Gaussian Smoothing and Regridding Pipeline for JWST/MIRI Line Cubes

This notebook implements a Gaussian-based smoothing workflow for JWST/MIRI–MRS line cubes.  
It smooths each spectral slice to the spatial resolution of the longest-wavelength line, regrids the cubes to a common spatial grid, applies masking, and preserves accurate WCS/header information (which CASA often alters).

If you need help generating the CASA line subcubes used as input, see:  
https://github.com/siwakotiutsav/casatools

---

## What the Code Does

### 1. Gaussian PSF-matching
For each wavelength slice:
- Computes the expected MIRI PSF FWHM  
  \(\theta(\lambda) = 0.033\,\lambda + 0.106\) arcsec  
- Computes the target FWHM at the longest wavelength  
- Convolves each slice with a 2D Gaussian kernel  
- Leaves slices unchanged if they already meet the target resolution

### 2. Regridding
- Uses a reference cube (typically the longest-wavelength line)
- Ensures all smoothed cubes share the same spatial grid
- Preserves WCS and header metadata accurately

### 3. Masking
- Applies a mask to remove NaNs
- Ensures consistent spatial coverage across all cubes

### 4. Output
- Writes each smoothed cube to the output directory
- Appends `_smooth.fits` to filenames
- Maintains original metadata

---

## Before Running

Make sure:
- `IN_DIR` contains your CASA-generated line subcubes  
- `REF_CUBE` points to the longest-wavelength line cube  
- `OUT_DIR` exists or can be created  
- All directory paths in the USER PARAMETERS section are correct  

If the paths are incorrect, the script will not find any files.

---

## Usage

Run the notebook cells in order.  
All smoothed cubes will appear in the directory specified by `OUT_DIR`.

---

## Notes on Methodology

This Gaussian-smoothing approach follows the method used in early JWST/MIRI–MRS studies, where Gaussian kernels approximate the wavelength-dependent PSF.

If you use this workflow in a publication:
- Cite the JWST/MIRI papers that introduced this smoothing method.
- Acknowledging this notebook or repository is appreciated but not required.

Example acknowledgment:

> “We used a publicly available Gaussian PSF-matching and regridding pipeline (Siwakoti 2025, github.com/siwakotiutsav/gaussian-convolution).”

---

## Related Tools

CASA-based line-cube extraction pipeline:  
https://github.com/siwakotiutsav/casatools


In [9]:
import scipy
import pylab
import os
import gc
import pprint
import numpy as np
import pandas as pd
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.io.fits as fits
from astropy import units as u
from astropy.wcs import WCS
import pyspeckit
import analysis as an
import pyneb as pn
from spectral_cube import SpectralCube 
import sklearn
from sklearn.decomposition import PCA
from reproject import reproject_interp
from astropy.convolution import convolve, Gaussian2DKernel



%matplotlib inline
                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # LaTeX:
matplotlib.rc('text', usetex=True)
#matplotlib.rc('font', family='sans-serif')
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)
#if not working: in terminal where launch jupyter notebook run
#export PATH=$PATH:/Library/TeX/texbin/latex

## Smoothing and regriding the line file

In [10]:
import glob

# ---------- USER PARAMETERS ----------
IN_DIR  = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic"  # folder containing MIRI .fits cubes
OUT_DIR = "../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth"   # output folder
REF_CUBE = "../../github/astr796_25_V2/NGC253_output/3channel/casa/atomic/NGC253_PIII_17.9.line.fits" # Fe reference
TARGET_LAMBDA = 17.8846  # µm
FILE_GLOB = os.path.join(IN_DIR, "*.fits")
# ------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)
files = sorted(glob.glob(FILE_GLOB))

if len(files) == 0:
    raise RuntimeError(f"No FITS cubes found in {IN_DIR}")

# --- Load reference cube wavelength grid ---
with fits.open(REF_CUBE) as ref:
    hdr_ref = ref[0].header
    naxis3 = hdr_ref["NAXIS3"]
    lam_ref = np.arange(naxis3) * hdr_ref["CDELT3"] + hdr_ref["CRVAL3"]

target_fwhm = 0.033 * TARGET_LAMBDA + 0.106  # arcsec

for infile in files:
    print(f"Processing: {os.path.basename(infile)}")
    sc = SpectralCube.read(infile, hdu=1)
    data = fits.getdata(infile, hdu=1)
    hdr = fits.getheader(infile, hdu=1)

    n_spec = hdr["NAXIS3"]
    lam = np.arange(n_spec) * hdr["CDELT3"] + hdr["CRVAL3"]

    pixscale = abs(hdr["CDELT2"]) * 3600.0  # arcsec/pix
    smoothed = np.zeros_like(data)

    mask = np.ones_like(data[0])
    mask[np.isnan(data[0])] = np.nan

    for i, lam_i in enumerate(lam):
        theta = 0.033 * lam_i + 0.106  # arcsec
        fwhm = np.sqrt(np.clip(target_fwhm**2 - theta**2, 0, None))
        sigma_pix = fwhm / 2.354 / pixscale

        if sigma_pix > 0:
            kernel = Gaussian2DKernel(x_stddev=sigma_pix)
            smoothed[i] = convolve(data[i], kernel)
        else:
            smoothed[i] = data[i]

    outname = os.path.join(OUT_DIR, os.path.basename(infile).replace(".fits", "_smooth.fits"))
    fits.writeto(outname, smoothed * mask, hdr, overwrite=True)
    print(f" → Saved: {outname}")

print("All cubes smoothed successfully.")


Processing: NGC253_ArIII_9.0.cont.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_ArIII_9.0.cont_smooth.fits
Processing: NGC253_ArIII_9.0.line.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_ArIII_9.0.line_smooth.fits
Processing: NGC253_ArII_7.0.cont.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_ArII_7.0.cont_smooth.fits
Processing: NGC253_ArII_7.0.line.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_ArII_7.0.line_smooth.fits
Processing: NGC253_C2H2_13.7.cont.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_C2H2_13.7.cont_smooth.fits
Processing: NGC253_C2H2_13.7.line.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth/NGC253_C2H2_13.7.line_smooth.fits
Processing: NGC253_CO2_15.0.cont.fits
 → Saved: ../../github/astr796_25_V2/NGC253_output/3channel/casa/l

In [11]:
## Trying to correctly pass the header:


# --- paths ---
smooth_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/line_smooth"
out_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid"
os.makedirs(out_dir, exist_ok=True)

# Reference cube (Fe II or ch4)
ref_path = os.path.join(smooth_dir, "NGC253_PIII_17.9.line_smooth.fits")
with fits.open(ref_path) as ref_hdul:
    ref_hdr = ref_hdul[0].header.copy()
    ref_data = ref_hdul[0].data.copy()
ref_wcs_3d = WCS(ref_hdr)
ref_wcs_2d = ref_wcs_3d.dropaxis(2)
ref_hdr_2d = ref_wcs_2d.to_header()
ny_ref, nx_ref = ref_data.shape[1], ref_data.shape[2]
shape_out_ref = (ny_ref, nx_ref)

# --- Input cubes ---
cube_files = sorted(glob.glob(os.path.join(smooth_dir, "*.fits")))
print(f"Found {len(cube_files)} smoothed cubes.")

for infile in cube_files:
    name = os.path.basename(infile)
    outfile = os.path.join(out_dir, name.replace(".fits", "_regrid.fits"))

    # --- Read input cube ---
    data_in = fits.getdata(infile)
    hdr_in = fits.getheader(infile)
    wcs_in_3d = WCS(hdr_in)
    wcs_in_2d = wcs_in_3d.dropaxis(2)
    hdr_in_2d = wcs_in_2d.to_header()

    nlam = data_in.shape[0]
    print(f"Regridding {name}: {nlam} slices -> {shape_out_ref}")

    # --- Reproject slice by slice ---
    regridded = np.empty((nlam, ny_ref, nx_ref), dtype=np.float32)
    for i in range(nlam):
        plane = data_in[i].astype(float)
        plane_masked = np.where(np.isfinite(plane), plane, np.nan)
        out_plane, _ = reproject_interp((plane_masked, hdr_in_2d),
                                        ref_hdr_2d, shape_out=shape_out_ref)
        regridded[i] = out_plane

    # --- Build output header ---
    out_hdr = ref_hdr.copy()
    out_hdr["NAXIS3"] = nlam
    for key in hdr_in:
        if "3" in key or key.startswith("REST") or key.startswith("SPECSYS"):
            out_hdr[key] = hdr_in[key]



    fits.writeto(outfile, regridded, out_hdr, overwrite=True)
    print(f"  → wrote {outfile} shape={regridded.shape}")

print("All smoothed cubes regridded successfully.")



Found 100 smoothed cubes.
Regridding NGC253_ArIII_9.0.cont_smooth.fits: 53 slices -> (93, 75)
  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid/NGC253_ArIII_9.0.cont_smooth_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArIII_9.0.line_smooth.fits: 53 slices -> (93, 75)
  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid/NGC253_ArIII_9.0.line_smooth_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArII_7.0.cont_smooth.fits: 67 slices -> (93, 75)
  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid/NGC253_ArII_7.0.cont_smooth_regrid.fits shape=(67, 93, 75)
Regridding NGC253_ArII_7.0.line_smooth.fits: 67 slices -> (93, 75)
  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid/NGC253_ArII_7.0.line_smooth_regrid.fits shape=(67, 93, 75)
Regridding NGC253_C2H2_13.7.cont_smooth.fits: 42 slices -> (93, 75)
  → wrote ../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid/NGC253_C2H2_1

In [12]:

# ---- Paths ----
input_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/line_regrid"          #use this for frequency based x axis
output_dir = "../../github/astr796_25_V2/NGC253_output/3channel/casa/line_masked"

os.makedirs(output_dir, exist_ok=True)

# ---- Reference Ch I cube for mask ----
ch1_ref = os.path.join(input_dir, "NGC253_S9_5.0.line_smooth_regrid.fits")         #for frequency based maps
if not os.path.exists(ch1_ref):
    raise FileNotFoundError(f"Ch 1 reference cube not found: {ch1_ref}")

fe_data = fits.getdata(ch1_ref)
if fe_data.ndim != 3:
    raise RuntimeError(f"Ch I cube not 2D: shape={fe_data.shape}")

mask = np.ones_like(fe_data[0], dtype=float)
mask[~np.isfinite(fe_data[0])] = np.nan
print(f"Mask built from Ch I first plane: shape={mask.shape}")

# ---- Apply mask to each cube ----
for fname in sorted(os.listdir(input_dir)):
    if not fname.endswith(".fits"):
        continue
    infile = os.path.join(input_dir, fname)
    data = fits.getdata(infile)
    hdr = fits.getheader(infile)

    if data.ndim != 3:
        print(f"Skipping {fname} (not 2D)")
        continue

    # Apply mask: NaN where Ch I is invalid
    data_masked = data * mask[np.newaxis, :, :]

    # Write masked cube
    outfile = os.path.join(output_dir, fname.replace(".fits", "_maskch1.fits"))
    fits.writeto(outfile, data_masked, hdr, overwrite=True)
    print(f"→ Masked {fname}  →  {os.path.basename(outfile)}")

print("All cubes masked using ch1 S9 reference.")


Mask built from Ch I first plane: shape=(93, 75)
→ Masked NGC253_ArIII_9.0.cont_smooth_regrid.fits  →  NGC253_ArIII_9.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ArIII_9.0.line_smooth_regrid.fits  →  NGC253_ArIII_9.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_ArII_7.0.cont_smooth_regrid.fits  →  NGC253_ArII_7.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ArII_7.0.line_smooth_regrid.fits  →  NGC253_ArII_7.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_C2H2_13.7.cont_smooth_regrid.fits  →  NGC253_C2H2_13.7.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_C2H2_13.7.line_smooth_regrid.fits  →  NGC253_C2H2_13.7.line_smooth_regrid_maskch1.fits
→ Masked NGC253_CO2_15.0.cont_smooth_regrid.fits  →  NGC253_CO2_15.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_CO2_15.0.line_smooth_regrid.fits  →  NGC253_CO2_15.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_ClII_14.4.cont_smooth_regrid.fits  →  NGC253_ClII_14.4.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ClII_14.4.line_s

## Code I might require for checks/ error in workflow, these may contain my earlier version of codes